In [28]:
# Cell 1: Core imports and path setup
import os
import sys
import json
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Tuple

# Add src to path for imports
sys.path.insert(0, os.path.abspath('.'))

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Basic imports successful")
print(f"Python version: {sys.version}")
print(f"Current directory: {os.getcwd()}")

✅ Basic imports successful
Python version: 3.12.11 (main, Aug  8 2025, 17:06:48) [Clang 20.1.4 ]
Current directory: /home/jroberts/fon_proposal_writer


In [29]:
# Cell 2: Load and verify environment variables
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Check critical environment variables
env_vars = {
    "AZURE_API_KEY": os.getenv("AZURE_API_KEY"),
    "AZURE_API_BASE": os.getenv("AZURE_API_BASE"),
    "AZURE_OPENAI_DEPLOYMENT": os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    "LANGFUSE_PUBLIC_KEY": os.getenv("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.getenv("LANGFUSE_SECRET_KEY"),
    "AZURE_STORAGE_CONNECTION_STRING": os.getenv("AZURE_STORAGE_CONNECTION_STRING"),
}

# Display status (hide actual values)
for key, value in env_vars.items():
    status = "✅ Set" if value else "❌ Missing"
    print(f"{key}: {status}")
    
missing = [k for k, v in env_vars.items() if not v]
if missing:
    print(f"\n⚠️ Missing environment variables: {', '.join(missing)}")
else:
    print("\n✅ All required environment variables are set")

AZURE_API_KEY: ✅ Set
AZURE_API_BASE: ✅ Set
AZURE_OPENAI_DEPLOYMENT: ✅ Set
LANGFUSE_PUBLIC_KEY: ✅ Set
LANGFUSE_SECRET_KEY: ✅ Set
AZURE_STORAGE_CONNECTION_STRING: ✅ Set

✅ All required environment variables are set


In [30]:
# Cell 3: Test configuration module
from src.config import settings

print("Configuration loaded:")
print(f"  Azure Deployment: {settings.azure_openai_deployment}")
print(f"  Langfuse Host: {settings.langfuse_host}")
print(f"  Blob Container: {settings.azure_blob_container}")
print(f"  Debug Mode: {settings.debug}")

# Verify Azure OpenAI endpoint construction
endpoint = f"{settings.azure_api_base}/openai/v1/"
print(f"\n  Constructed endpoint: {endpoint}")

Configuration loaded:
  Azure Deployment: gpt-4.1
  Langfuse Host: https://us.cloud.langfuse.com
  Blob Container: rfp-poc
  Debug Mode: False

  Constructed endpoint: https://proposal-openai-model.openai.azure.com//openai/v1/


In [69]:
import os
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

PROPOSAL_ZIP = "Senate_Capitol_Facilities"

def download_zip_file():
    """Download zip file from Azure Blob Storage"""
    
    # Initialize the BlobServiceClient using connection string
    blob_service_client = BlobServiceClient.from_connection_string(
        os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    )
    
    # You need to know the container name - common names are:
    # "data", "files", "uploads", "rfp-poc", etc.
    container_name = "proposal-container"  # Replace with your actual container name
    blob_name = f"{PROPOSAL_ZIP}.zip"
    local_file_path = f"./data/{PROPOSAL_ZIP}.zip"  # Where to save locally
    
    try:
        # Get blob client
        blob_client = blob_service_client.get_blob_client(
            container=container_name, 
            blob=blob_name
        )
        
        # Download the blob
        print(f"Downloading {blob_name}...")
        with open(local_file_path, "wb") as download_file:
            download_data = blob_client.download_blob()
            download_file.write(download_data.readall())
        
        print(f"✅ Successfully downloaded to {local_file_path}")
        
    except Exception as e:
        print(f"❌ Error downloading file: {e}")
        
        # If container name is wrong, list available containers
        if "ContainerNotFound" in str(e):
            print("\nAvailable containers:")
            for container in blob_service_client.list_containers():
                print(f"  - {container.name}")

# Run the download
download_zip_file()

2025-09-03 12:01:25,107 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://proposalaistorage.blob.core.windows.net/proposal-container/Senate_Capitol_Facilities.zip'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.26.0 Python/3.12.11 (Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '3e32a9e0-88df-11f0-8e45-4b1764a8500a'
    'Authorization': 'REDACTED'
No body was attached to the request
2025-09-03 12:01:25,308 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 206
Response headers:
    'Content-Length': '1974880'
    'Content-Type': 'application/x-zip-compressed'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Wed, 03 Sep 2025 15:04:15 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DDEAFB264B0A40"'
    'Server': 'Wind

✅ Successfully downloaded to ./data/Senate_Capitol_Facilities.zip


In [70]:
import zipfile

# Unzip the file to current directory
with zipfile.ZipFile(f'./data/{PROPOSAL_ZIP}.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/inputs/')  # '.' means current directory

print("✅ Unzipped all files to current directory")

✅ Unzipped all files to current directory


In [64]:
# Cell 4: Test PDF loader
from src.io.loaders import pdf_to_pages

# Create a test PDF or use an existing one
test_pdf_path = "data/inputs/DOE_Personnel/Attachment 1 - Administrative Background Investigation Support Services - PWS.1740156524666 (1).pdf"

if Path(test_pdf_path).exists():
    pages = pdf_to_pages(test_pdf_path)
    print(f"✅ Successfully loaded PDF: {test_pdf_path}")
    print(f"  Number of pages: {len(pages)}")
    # print(f"  First page preview (first 500 chars):")
    # if pages:
    #     print(f"  {pages[0][1][:500]}...")
else:
    print(f"❌ Test PDF not found at: {test_pdf_path}")
    print("  Create a test PDF or update the path")

✅ Successfully loaded PDF: data/inputs/DOE_Personnel/Attachment 1 - Administrative Background Investigation Support Services - PWS.1740156524666 (1).pdf
  Number of pages: 10


In [65]:
# Cell 5: Convert pages to chunk format (no chunking, just formatting)
# Skip the chunking step - use pages directly
page_chunks = []
for page_num, page_text in pages:
    page_chunk = {
        "text": page_text,
        "section": f"Page {page_num}",  # Simple section identifier
        "start_page": page_num,
        "end_page": page_num
    }
    page_chunks.append(page_chunk)

print(f"✅ Created {len(page_chunks)} page chunks (1 per page)")

✅ Created 10 page chunks (1 per page)


In [34]:
# Cell 6: Run regex on full pages
from src.preprocessing.regex_pass import fast_hits

all_matches = []
for page_chunk in page_chunks:
    matches = fast_hits(page_chunk)
    all_matches.extend(matches)
    
print(f"✅ Found {len(all_matches)} total regex matches across all pages")

✅ Found 0 total regex matches across all pages


In [62]:
# Cell 7: Initialize DSPy with Azure OpenAI
import dspy
from src.config import settings

# Configure DSPy
lm = dspy.LM(
    model=settings.azure_openai_deployment,
    api_key=settings.azure_api_key,
    api_base=f"{settings.azure_api_base}/openai/v1/",
    temperature=0.1,
    max_tokens=32700
)

# Set up with JSON adapter for structured output
dspy.configure(lm=lm, adapter=dspy.JSONAdapter(), track_usage=False, cache=True)
# dspy.configure_cache(
#     enable_disk_cache=False,
#     enable_memory_cache=False,
# )

print(f"✅ DSPy configured with model: azure/{settings.azure_openai_deployment}")

# # Test with a simple prompt
# test_lm = dspy.Predict("question -> answer")
# try:
#     result = test_lm(question="What is 5+3?")
#     print(f"✅ DSPy test successful: {result.answer}")
# except Exception as e:
#     print(f"❌ DSPy test failed: {e}")
#     print("API Base:", settings.azure_api_base)
#     print("Deployment:", settings.azure_openai_deployment)
#     print("API Key set:", bool(settings.azure_api_key))

✅ DSPy configured with model: azure/gpt-4.1


In [63]:
# Cell 8: Initialize Langfuse tracing
from src.observability.tracing import initialize_tracing, get_langfuse_client

try:
    langfuse_client = initialize_tracing()
    print("✅ Langfuse tracing initialized")
    print(f"  Host: {settings.langfuse_host}")
    
    # Create a test trace
    from langfuse import observe
    
    @observe(name="test_function")
    def test_trace():
        return "Test successful"
    
    result = test_trace()
    langfuse_client.flush()
    print(f"✅ Test trace created: {result}")
    
except Exception as e:
    print(f"⚠️ Langfuse initialization warning: {e}")
    print("  Continuing without tracing...")

✅ Langfuse tracing initialized
  Host: https://us.cloud.langfuse.com
✅ Test trace created: Test successful


In [66]:
# Cell 9: Process pages through language model (process specific pages or all)
from src.extraction.modules import Extractor
import json

extractor = Extractor()

# Process specific pages (e.g., last 10 pages which often have requirements)
pages_to_process = page_chunks  # Last 10 pages, or use page_chunks[:10] for first 10
print(f"Processing {len(pages_to_process)} pages (pages {pages_to_process[0]['start_page']}-{pages_to_process[-1]['end_page']})")

all_extracted = []
for i, page_chunk in enumerate(pages_to_process):
    print(f"Processing page {page_chunk['start_page']} ({i+1}/{len(pages_to_process)})")
    try:
        requirements = extractor(page_chunk)
        all_extracted.extend(requirements)
        print(f"  → Found {len(requirements)} requirements")
        for req in requirements[:2]:  # Show first 2 from each page
            print(f"    • {req.get('label', 'No label')[:80]}...")
    except Exception as e:
        print(f"  ⚠️ Extraction failed: {e}")

print(f"\n✅ Total extracted: {len(all_extracted)} requirements from {len(pages_to_process)} pages")

Processing 10 pages (pages 1-10)
Processing page 1 (1/10)
  → Found 6 requirements
    • Contractor must provide personnel security support services to the Personnel Sec...
    • Contractor must provide Team Lead (1 each) and Administrative Assistant (3 each)...
Processing page 2 (2/10)
  → Found 7 requirements
    • Contractor shall provide qualified and skilled personnel, supervision, equipment...
    • Contractor shall ensure all contractor employees are sufficiently qualified to p...
Processing page 3 (3/10)
  → Found 16 requirements
    • Provide expertise and perform all necessary administrative services for NR Perso...
    • Perform vault and PSF maintenance, clearance request processing, complete securi...
Processing page 4 (4/10)
  → Found 11 requirements
    • Provide expertise and perform all necessary services to train personnel to perfo...
    • All personnel shall meet training requirements for processing, and complete refr...
Processing page 5 (5/10)
  → Found 17 require

2025-09-03 22:14:28,477 - src.extraction.modules - ERROR - Failed to parse JSON from extractor: Expecting value: line 1 column 2 (char 1)


  → Found 0 requirements
Processing page 8 (8/10)
  → Found 10 requirements
    • Provide timely notification (within 24 hours) of any deviation from records mana...
    • All data entered into CATS, budget continuous evaluation or successor databases ...
Processing page 9 (9/10)
  → Found 3 requirements
    • Provide 100% personnel coverage for personnel security services during core hour...
    • Provide vault coverage (open and close) between specified hours...
Processing page 10 (10/10)
  → Found 22 requirements
    • Compliance with Title 10 CFR Part 851 – Worker Safety and Health Program...
    • Compliance with Title 48 CFR Part 22 – Application of Labor Laws to Government A...

✅ Total extracted: 98 requirements from 10 pages


In [35]:
import dspy

# See what methods the cache has
print("Cache methods and attributes:")
for attr in dir(dspy.cache):
    if not attr.startswith('_'):
        print(f"  {attr}")

# Check for common cache patterns
if hasattr(dspy.cache, 'cache'):
    print("\nFound cache.cache - type:", type(dspy.cache.cache))
    if hasattr(dspy.cache.cache, 'clear'):
        dspy.cache.cache.clear()
        print("✓ Cleared dspy.cache.cache")

if hasattr(dspy.cache, '_cache'):
    print("\nFound cache._cache - type:", type(dspy.cache._cache))
    if hasattr(dspy.cache._cache, 'clear'):
        dspy.cache._cache.clear()
        print("✓ Cleared dspy.cache._cache")

if hasattr(dspy.cache, 'data'):
    print("\nFound cache.data - type:", type(dspy.cache.data))
    if hasattr(dspy.cache.data, 'clear'):
        dspy.cache.data.clear()
        print("✓ Cleared dspy.cache.data")

# Try to see the internal structure
print("\nAll cache attributes (including private):")
for attr in dir(dspy.cache):
    try:
        value = getattr(dspy.cache, attr)
        if not callable(value) and not attr.startswith('__'):
            print(f"  {attr}: {type(value)}")
    except:
        pass

Cache methods and attributes:
  cache_key
  disk_cache
  enable_disk_cache
  enable_memory_cache
  get
  load_memory_cache
  memory_cache
  put
  reset_memory_cache
  save_memory_cache

All cache attributes (including private):
  _lock: <class '_thread.RLock'>
  disk_cache: <class 'diskcache.fanout.FanoutCache'>
  enable_disk_cache: <class 'bool'>
  enable_memory_cache: <class 'bool'>
  memory_cache: <class 'cachetools.LRUCache'>


In [39]:
print(f"Memory cache size: {len(dspy.cache.memory_cache)}")
print(f"Disk cache size: {len(dspy.cache.disk_cache)}")


# 1. Clear the memory cache
dspy.cache.reset_memory_cache()
print("✓ Memory cache reset")

# 2. Clear the disk cache
dspy.cache.disk_cache.clear()
print("✓ Disk cache cleared")

print(f"Memory cache size: {len(dspy.cache.memory_cache)}")
print(f"Disk cache size: {len(dspy.cache.disk_cache)}")


Memory cache size: 1
Disk cache size: 1
✓ Memory cache reset
✓ Disk cache cleared
Memory cache size: 0
Disk cache size: 0


In [53]:
test_type = "extractor_rfp_letter"
test_number = 3
with open(f"./test-results/{test_type}_test_{str(test_number)}.json", "x") as f:
    json.dump(all_extracted, f, indent=2)

print(f"Extracted requirements saved to ./test-results/{test_type}_{str(test_number)}.json")

Extracted requirements saved to ./test-results/extractor_rfp_letter_3.json


In [20]:
import json
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment

def json_to_excel_simple(json_file_path, excel_file_path):
    """
    Convert JSON file to Excel - simple and clean.
    """
    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Write to Excel
    df.to_excel(excel_file_path, index=False, sheet_name='Requirements')
    
    print(f"✅ Converted {json_file_path} to {excel_file_path}")
    print(f"📊 {len(data)} rows exported")
    return df

def json_to_excel_formatted(json_file_path, excel_file_path):
    """
    Convert JSON file to Excel with nice formatting.
    """
    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Write to Excel with formatting
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Requirements')
        
        # Get the workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets['Requirements']
        
        # Format headers
        header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
        header_font = Font(bold=True, color='FFFFFF')
        
        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Auto-adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            
            # Set reasonable max width
            adjusted_width = min(max_length + 2, 50)
            worksheet.column_dimensions[column_letter].width = adjusted_width
        
        # Wrap text for long content columns
        for row in worksheet.iter_rows(min_row=2):
            for cell in row:
                if cell.column_letter in ['B', 'E']:  # label and quote columns
                    cell.alignment = Alignment(wrap_text=True, vertical='top')
    
    print(f"✅ Converted {json_file_path} to {excel_file_path} (with formatting)")
    print(f"📊 {len(data)} rows exported")
    return df


# Simple conversion
print("🚀 Simple conversion:")
df = json_to_excel_simple('./test-results/extractor_test_2.json', 'requirements_simple.xlsx')

# Formatted conversion
print("\n✨ Formatted conversion:")
df = json_to_excel_formatted('./test-results/extractor_test_2.json', 'requirements_formatted.xlsx')

🚀 Simple conversion:
✅ Converted ./test-results/extractor_test_2.json to requirements_simple.xlsx
📊 29 rows exported

✨ Formatted conversion:
✅ Converted ./test-results/extractor_test_2.json to requirements_formatted.xlsx (with formatting)
📊 29 rows exported


In [54]:
# Cell 10: Classify all extracted requirements
from src.extraction.modules import Classifier

classifier = Classifier()
classified_reqs = []

print(f"Classifying {len(all_extracted)} requirements...\n")

for i, req in enumerate(all_extracted):
    try:
        classified = classifier(req)
        classified_reqs.append(classified)
        if (i % 5) == 0:  # Show first 5 classifications
            print(f"Requirement {i+1}:{len(all_extracted)}")
            # print(f"  Label: {classified.get('label', 'Unknown')[:80]}...")
            # print(f"  Category: {classified.get('category', 'Unknown')}")
            # print(f"  Modality: {classified.get('modality', 'Unknown')}\n")
    except Exception as e:
        print(f"  ⚠️ Classification failed for requirement {i+1}: {e}")
        classified_reqs.append(req)  # Keep original if classification fails

print(f"✅ Classified {len(classified_reqs)} requirements")

# Show distribution
from collections import Counter
categories = Counter(r.get('category', 'Unknown') for r in classified_reqs)
modalities = Counter(r.get('modality', 'Unknown') for r in classified_reqs)

print("\nCategory distribution:")
for cat, count in categories.most_common():
    print(f"  {cat}: {count}")

print("\nModality distribution:")
for mod, count in modalities.most_common():
    print(f"  {mod}: {count}")

Classifying 371 requirements...

Requirement 1:371
Requirement 6:371
Requirement 11:371
Requirement 16:371
Requirement 21:371
Requirement 26:371
Requirement 31:371
Requirement 36:371
Requirement 41:371
Requirement 46:371
Requirement 51:371
Requirement 56:371
Requirement 61:371
Requirement 66:371
Requirement 71:371
Requirement 76:371
Requirement 81:371
Requirement 86:371
Requirement 91:371
Requirement 96:371
Requirement 101:371
Requirement 106:371
Requirement 111:371
Requirement 116:371
Requirement 121:371
Requirement 126:371
Requirement 131:371
Requirement 136:371
Requirement 141:371
Requirement 146:371
Requirement 151:371
Requirement 156:371
Requirement 161:371
Requirement 166:371
Requirement 171:371
Requirement 176:371
Requirement 181:371
Requirement 186:371
Requirement 191:371
Requirement 196:371
Requirement 201:371
Requirement 206:371
Requirement 211:371
Requirement 216:371
Requirement 221:371
Requirement 226:371
Requirement 231:371
Requirement 236:371
Requirement 241:371
Requireme

In [55]:
test_type = "classifier_reqs_rfp_letter.json"
test_number = 3
with open(f"./test-results/{test_type}_test_{str(test_number)}.json", "x") as f:
    json.dump(classified_reqs, f, indent=2)

print(f"Extracted requirements saved to ./test-results/{test_type}_{str(test_number)}.json")

Extracted requirements saved to ./test-results/classifier_reqs_rfp_letter.json_3.json


In [56]:
print(len(classified_reqs))
print(len(all_extracted))

371
371


In [45]:
# Cell 11: Ground requirements with evidence from source chunks
from src.extraction.modules import Grounder

grounder = Grounder()
grounded_reqs = []

# Map requirements back to their source chunks
print(f"Grounding {len(classified_reqs)} requirements with evidence...\n")

for i, req in enumerate(classified_reqs):
    # Find the source chunk (assuming requirements maintain order)
    chunk_index = min(i // 2, len(pages_to_process) - 1)  # Rough mapping
    source_chunk = pages_to_process[chunk_index]
    
    try:
        grounded = grounder(source_chunk, req)
        grounded_reqs.append(grounded)
        if (i % 3) == 0:  # Show first 3 grounded requirements
            print(f"Requirement {i+1}:")
            # print(f"  Label: {grounded.get('label', 'Unknown')[:60]}...")
            # print(f"  Evidence: {grounded.get('quote', 'No quote')[:100]}...")
            # print(f"  Pages: {grounded.get('page_start', '?')}-{grounded.get('page_end', '?')}\n")
    except Exception as e:
        print(f"  ⚠️ Grounding failed for requirement {i+1}: {e}")
        grounded_reqs.append(req)

# Add remaining requirements without grounding
grounded_reqs.extend(classified_reqs[20:])

print(f"✅ Grounded {len(grounded_reqs)} requirements")

Grounding 246 requirements with evidence...

Requirement 1:
Requirement 4:
Requirement 7:
Requirement 10:
Requirement 13:
Requirement 16:
Requirement 19:
Requirement 22:
Requirement 25:
Requirement 28:
Requirement 31:
Requirement 34:
Requirement 37:
Requirement 40:
Requirement 43:
Requirement 46:
Requirement 49:
Requirement 52:
Requirement 55:
Requirement 58:
Requirement 61:
Requirement 64:
Requirement 67:
Requirement 70:
Requirement 73:
Requirement 76:
Requirement 79:
Requirement 82:
Requirement 85:
Requirement 88:
Requirement 91:
Requirement 94:
Requirement 97:
Requirement 100:
Requirement 103:
Requirement 106:
Requirement 109:
Requirement 112:
Requirement 115:
Requirement 118:
Requirement 121:
Requirement 124:
Requirement 127:
Requirement 130:
Requirement 133:
Requirement 136:
Requirement 139:
Requirement 142:
Requirement 145:
Requirement 148:
Requirement 151:
Requirement 154:
Requirement 157:
Requirement 160:
Requirement 163:
Requirement 166:
Requirement 169:
Requirement 172:
Requi

In [46]:
test_type = "grounder"
test_number = 3
with open(f"./test-results/{test_type}_test_{str(test_number)}.json", "x") as f:
    json.dump(all_extracted, f, indent=2)

print(f"Extracted requirements saved to ./test-results/{test_type}_{str(test_number)}.json")

test_type = "grounder_reqs"
test_number = 3
with open(f"./test-results/{test_type}_test_{str(test_number)}.json", "x") as f:
    json.dump(grounded_reqs, f, indent=2)

print(f"Extracted requirements saved to ./test-results/{test_type}_{str(test_number)}.json")

Extracted requirements saved to ./test-results/grounder_3.json
Extracted requirements saved to ./test-results/grounder_reqs_3.json


In [61]:
# Test your existing src/extraction/merge_dedupe.py with confidence normalization

import json, csv, re
from pathlib import Path
from typing import List, Dict, Any
from src.extraction.merge_dedupe import merge_dedupe

# <<< EDIT THESE >>>
INPUT_FILES = [
    "test-results/classifier_reqs_rfp_letter.json_test_3.json",
    "test-results/classifier_reqs_test_3.json",
]
OUT_JSON = "canonical_requirements.json"
OUT_CSV  = "canonical_requirements.csv"

def _read_any_json(path: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        for key in ("requirements", "items", "data", "records", "rows"):
            if key in data and isinstance(data[key], list):
                return data[key]
    return [data]

def _to_confidence(v) -> float | None:
    if v is None: 
        return None
    if isinstance(v, (int, float)): 
        return float(v)
    if isinstance(v, bool):
        return 1.0 if v else 0.0
    if isinstance(v, str):
        s = v.strip().lower()
        # percent "85%"
        if s.endswith("%"):
            try:
                return float(s[:-1]) / 100.0
            except Exception:
                return None
        # numeric-in-string "0.92" / "1"
        try:
            return float(s)
        except Exception:
            pass
        # labels
        if s in {"very high","vh"}: return 0.99
        if s in {"high","h","strong"}: return 0.95
        if s in {"medium","med","moderate"}: return 0.65
        if s in {"low","l","weak"}: return 0.35
        if s in {"very low","vl"}: return 0.15
    return None

def _coerce_for_merge(records: List[Dict[str, Any]], default_doc: str) -> List[Dict[str, Any]]:
    out = []
    for rec in records:
        if not isinstance(rec, dict):
            continue
        r = dict(rec)  # shallow copy
        # Source/doc
        r["doc"] = r.get("doc") or r.get("source") or default_doc
        r["source"] = r.get("source") or r["doc"]
        # Fields merge_dedupe expects
        r["label"] = r.get("label") or r.get("title") or r.get("requirement_label") or ""
        r["quote"] = r.get("quote") or r.get("text") or r.get("requirement_text") or r.get("chunk_text") or ""
        r["category"] = r.get("category") or r.get("topic") or ""
        r["section"]  = r.get("section") or r.get("heading") or r.get("section_path")
        if "page_start" not in r or r.get("page_start") is None:
            r["page_start"] = r.get("page") or r.get("page_no")
        # Normalize confidence -> float
        r["confidence"] = _to_confidence(r.get("confidence"))
        out.append(r)
    return out

# Load + coerce (including confidence normalization)
rows: List[Dict[str, Any]] = []
for p in INPUT_FILES:
    p = Path(p)
    rows.extend(_coerce_for_merge(_read_any_json(str(p)), default_doc=p.stem))

before = len(rows)
canon  = merge_dedupe(rows)
after  = len(canon)

# Save outputs
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(canon, f, ensure_ascii=False, indent=2)

# Lightweight CSV (no pandas required)
fieldnames = sorted({k for r in canon for k in r.keys()})
if "sources" not in fieldnames:
    fieldnames.append("sources")
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    for r in canon:
        row = dict(r)
        row["sources"] = json.dumps(r.get("sources", []), ensure_ascii=False)
        w.writerow(row)

print("=== Dedupe Summary ===")
print(f"Input files           : {len(INPUT_FILES)}")
print(f"Total input records   : {before}")
print(f"Final canonical count : {after}")
print(f"Duplicates removed    : {before - after}")
print(f"Wrote JSON -> {OUT_JSON}")
print(f"Wrote CSV  -> {OUT_CSV}")

# Optional: quick look at which items merged multiple sources
top = sorted(canon, key=lambda x: len(x.get("sources", [])), reverse=True)[:15]
print("\nTop merged (id, sources_count, label snippet):")
for r in top:
    print(f"- {r.get('id')}  x{len(r.get('sources', []))}  :: {(r.get('label') or '')[:80]}")


=== Dedupe Summary ===
Input files           : 2
Total input records   : 617
Final canonical count : 577
Duplicates removed    : 40
Wrote JSON -> canonical_requirements.json
Wrote CSV  -> canonical_requirements.csv

Top merged (id, sources_count, label snippet):
- 8  x4  :: Contractor must propose qualified replacement for key personnel within 30 days o
- 4.5.5  x3  :: Contractor SHALL determine employee's options under FEHB.
- 6  x2  :: Present information in oral presentation or PowerPoint to facilitate evaluation 
- 4  x2  :: Clauses incorporated by reference
- D.1-1  x2  :: Initial and annual privacy training for employees with PII or system of records 
- 1  x2  :: Contractor must maintain and provide privacy training completion documentation u
- 2  x2  :: Employee access to PII or system of records requires completed privacy training.
- 3  x2  :: Include privacy training clause in all subcontracts where subcontractor employee
- 6  x2  :: Contractor must submit list of employee can

In [1]:
import json
from pathlib import Path
from typing import List, Dict, Any, Optional
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

PREFERRED_ORDER = [
    "id", "label", "category", "modality",
    "quote", "section", "page_start", "page_end", "confidence"
]

def load_json_array(path: str) -> List[Dict[str, Any]]:
    """Load a JSON file expected to contain a list[dict]."""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        if not isinstance(data, list):
            raise ValueError(f"{path} does not contain a JSON array.")
        return data

def union_columns(*rows_lists: List[List[Dict[str, Any]]]) -> List[str]:
    """Union of keys across all objects, with a friendly preferred order."""
    keys = set()
    for rows in rows_lists:
        for r in rows:
            keys.update(r.keys())
    # Put known keys first (if present), then any extras in alphabetical order
    extras = [k for k in sorted(keys) if k not in PREFERRED_ORDER]
    ordered = [k for k in PREFERRED_ORDER if k in keys] + extras
    return ordered

def drop_duplicates(rows: List[Dict[str, Any]], on: Optional[List[str]] = None) -> List[Dict[str, Any]]:
    """Optionally dedupe by one or more keys (e.g., ['id'] or ['quote'])."""
    if not on:
        return rows
    seen = set()
    out = []
    for r in rows:
        key = tuple(r.get(k) for k in on)
        if key in seen:
            continue
        seen.add(key)
        out.append(r)
    return out

def coerce_int(v: Any) -> Any:
    try:
        if v is None or v == "":
            return None
        # Only coerce if it's numeric-ish
        return int(v)
    except Exception:
        return v

def write_excel(rows: List[Dict[str, Any]], columns: List[str], out_path: str) -> Path:
    wb = Workbook()
    ws = wb.active
    ws.title = "requirements"

    # Header
    ws.append(columns)

    # Rows
    for r in rows:
        row = []
        for c in columns:
            val = r.get(c)
            if c in ("page_start", "page_end"):
                val = coerce_int(val)
            row.append(val)
        ws.append(row)

    # Tidy column widths (simple heuristic)
    for idx, col_name in enumerate(columns, start=1):
        max_len = len(col_name)
        for cell in ws[get_column_letter(idx)]:
            if cell.value is not None:
                max_len = max(max_len, len(str(cell.value)))
        ws.column_dimensions[get_column_letter(idx)].width = min(max_len + 2, 60)

    out = Path(out_path)
    wb.save(out)
    return out

def combine_json_to_excel(
    json_path_1: str,
    json_path_2: str,
    out_path: str = "combined.xlsx",
    dedupe_on: Optional[List[str]] = None,   # e.g., ["id"] or ["quote"]
) -> Path:
    rows1 = load_json_array(json_path_1)
    rows2 = load_json_array(json_path_2)

    all_rows = rows1 + rows2
    if dedupe_on:
        all_rows = drop_duplicates(all_rows, on=dedupe_on)

    cols = union_columns(all_rows)
    return write_excel(all_rows, cols, out_path)

output = combine_json_to_excel("test-results/DHS_CBP_RFP_Letter_classifier_reqs.json", "test-results/DHS_CBP_SOW_classifier_reqs.json", out_path="DHS_CBP_Requirements.xlsx")
print(f"✅ Wrote {output.resolve()}")

✅ Wrote /home/jroberts/fon_proposal_writer/DHS_CBP_Requirements.xlsx
